In [ ]:
pip install syft

In [ ]:
pip install matplotlib
#pip install --upgrade tensorflow

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy
from syft.frameworks.torch.fl import utils
import matplotlib.pyplot as plt 

In [2]:
class Parser:
    def __init__(self):
        self.epochs = 20
        self.lr = 0.001
        self.test_batch_size = 1000
        self.batch_size = 64
        self.log_interval = 10
        self.seed = 1
    
args = Parser()
torch.manual_seed(args.seed)

In [3]:
hook = sy.TorchHook(torch)  
bob = sy.VirtualWorker(hook, id="bob")  
alice = sy.VirtualWorker(hook, id="alice")  
nodes=[bob,alice]

In [4]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=False,
                   transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))]))
    #.federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    ,batch_size=args.batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

#sy.FederatedDataLoader- when we were using federate 
#note one (x,y) is : number of x's in one are based on batch size - same way for y

In [5]:
len(train_loader.dataset)
len(train_loader)

938

In [6]:
remote_dataset = (list(), list())

for batch_idx, (data,target) in enumerate(train_loader):
    data = data.send(nodes[batch_idx % len(nodes)])# can send using .federate directly and using FederatedDataLoader but that was giving issues so sending manually
    target = target.send(nodes[batch_idx % len(nodes)])
    remote_dataset[batch_idx % len(nodes)].append((data, target))

In [7]:
len(remote_dataset[0])
#x,y=remote_dataset[0][0]
#x

469

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [9]:
bobs_model = Net()
alices_model = Net()
bobs_optimizer = optim.Adam(bobs_model.parameters(), lr=args.lr)
alices_optimizer = optim.Adam(alices_model.parameters(), lr=args.lr)

models = [bobs_model, alices_model]
optimizers = [bobs_optimizer, alices_optimizer]

In [ ]:
global_model = Net()
print(global_model)
weights=global_model.fc2.weight.data  #gives last layer weights before softmax output
print(weights)
#plt.plot(weights)
#plt.show()
w = list(global_model.parameters()) #gives all parameters(weights of all)
print(w)

In [10]:
def update(x, y, model, optimizer):
    model.send(x.location)#sending to correct location
    optimizer.zero_grad()
    prediction = model(x)
    loss = F.nll_loss(prediction, y)
    loss.backward()
    optimizer.step()
    s = torch.sum(model.fc2.weight.data)
    print(s)
    return model

def train():
    for data_index in range(len(remote_dataset[0])-1):# 2 datasets (one w bob and other w alice)
        for worker_index in range(len(nodes)):
            x, y = remote_dataset[worker_index][data_index]
            models[worker_index] = update(x, y, models[worker_index], optimizers[worker_index])
        for model in models:
            model.get()
        #print(models[0])
        return utils.federated_avg({
            "bob": models[0],
            "alice": models[1]# securely aggregate both models and return global model
        })

In [ ]:
trained_weights_bob=models[0].fc2.weight.data  #last layer weights bob
plt.plot(trained_weights_bob)
plt.show()

In [11]:
def test(federated_model):
    federated_model.eval()
    test_loss = 0
    correct=0
    for x, y in test_loader:
        output = federated_model(x)
        test_loss += F.nll_loss(output, y, reduction='sum').item() # sum up batch loss
        pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
        #print(y.view_as(pred))
        #print(pred)
        correct += pred.eq(y.view_as(pred)).sum().item()
        #print(correct)
        
    test_loss /= len(test_loader.dataset)#this is total number of images in test data, whereas test_loader is z images grouped together depending on batch size,
    #so if batch size is 10 then 10 x's and 10 corresponding y's will go together thus y vector is like that
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
          (test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
    #print('Test set: Average loss: {:.4f}'.format(test_loss))
        

In [12]:
for epoch in range(args.epochs):
    print(f"Epoch Number {epoch + 1}")
    federated_model = train()
    global_model = federated_model
    test(federated_model)

Epoch Number 1
(Wrapper)>[PointerTensor | me:82276082999 -> bob:98796756249]
(Wrapper)>[PointerTensor | me:42106527622 -> alice:58915896887]

Test set: Average loss: 2.2822, Accuracy: 1855/10000 (19%)

Epoch Number 2
(Wrapper)>[PointerTensor | me:60750013013 -> bob:76193917730]
(Wrapper)>[PointerTensor | me:39717693624 -> alice:4279880527]

Test set: Average loss: 2.1763, Accuracy: 3096/10000 (31%)

Epoch Number 3
(Wrapper)>[PointerTensor | me:26378459524 -> bob:42818772738]
(Wrapper)>[PointerTensor | me:69514194504 -> alice:62145568709]

Test set: Average loss: 1.9386, Accuracy: 4072/10000 (41%)

Epoch Number 4
(Wrapper)>[PointerTensor | me:86651289767 -> bob:51565051279]
(Wrapper)>[PointerTensor | me:40687139788 -> alice:85367152746]

Test set: Average loss: 1.6433, Accuracy: 4981/10000 (50%)

Epoch Number 5
(Wrapper)>[PointerTensor | me:19874933939 -> bob:97110431828]
(Wrapper)>[PointerTensor | me:17167280398 -> alice:54964575391]

Test set: Average loss: 1.3595, Accuracy: 6247/1000